In [1]:
import os
import pandas as pd
import psycopg2
import re

def est_mobile(numero):
    motif_mobile = r'^(6\d{8}|\+336\d{8}|7\d{8})'
    return bool(re.match(motif_mobile, str(numero)))

def est_fixe(numero):
    motif_fixe = r'^(1\d{8}|2\d{8}|3\d{8}|4\d{8}|5\d{8})'
    return bool(re.match(motif_fixe, str(numero)))

def est_email(email):
    return re.match(r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$', email)

def insert_data_from_excel_folder(folder_path):
    try:
        dbname = "waouh_base"
        user = "waouhmonde"
        password = "waouhmonde"
        host = "localhost"
        port = "5432"

        # Connexion à la base de données
        with psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port) as conn:
            with conn.cursor() as cursor:
                try:
                    # Get the list of Excel files in the folder
                    excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]

                    for excel_file in excel_files:
                        excel_file_path = os.path.join(folder_path, excel_file)
                        print(f"Inserting data from {excel_file}...")

                        # Read Excel file into a dictionary of DataFrames
                        excel_data = pd.read_excel(excel_file_path, sheet_name=None)

                        for sheet_name, df in excel_data.items():
                            for _, row in df.iterrows():
                                nom = str(row['societe']).lower()
                                secteur_activite = row['secteur']
                                site_web = row['site']
                                valeur_tel1 = row['mobile1']
                                valeur_tel2 = row['mobile2']
                                valeur_email = row.get('mail', '')
                                adresse = str(row['adresse']).lower()
                                ville = str(row['commune']).lower()
                                opt_in = False
                                status_sms = False
                                status_appel = False
                                status_mail = True
                                source = "go-africa"

                                # Determine the type of contact based on the phone number
                                if est_mobile(valeur_tel1):
                                    type_contact_tel1 = 'Mobile'
                                elif est_fixe(valeur_tel1):
                                    type_contact_tel1 = 'Fixe'
                                else:
                                    type_contact_tel1 = ''

                                if est_mobile(valeur_tel2):
                                    type_contact_tel2 = 'Mobile'
                                elif est_fixe(valeur_tel2):
                                    type_contact_tel2 = 'Fixe'
                                else:
                                    type_contact_tel2 = ''

                                # Insert data into the Entreprises table
                                insert_query_entreprises = '''
                                    INSERT INTO Entreprises (
                                        nom, secteur_activite, site_web, adresse, ville, opt_in, status_sms, status_appel, status_mail
                                    )
                                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                                    RETURNING entreprise_id;
                                '''

                                data_tuple_entreprises = (nom, secteur_activite, site_web, adresse, ville, opt_in, status_sms, status_appel, status_mail)
                                cursor.execute(insert_query_entreprises, data_tuple_entreprises)
                                entreprise_id = cursor.fetchone()[0]

                                # Insert data into the Contacts_Entreprises table for phone numbers
                                insert_query_contacts_tel1 = '''
                                    INSERT INTO Contacts_Entreprises (
                                        entreprise_id, type, valeur, sources, status
                                    )
                                    VALUES (%s, %s, %s, %s, %s);
                                '''

                                data_tuple_contacts_tel1 = (entreprise_id, type_contact_tel1, valeur_tel1, source, True)
                                cursor.execute(insert_query_contacts_tel1, data_tuple_contacts_tel1)

                                if est_mobile(valeur_tel2) or est_fixe(valeur_tel2):
                                    insert_query_contacts_tel2 = '''
                                        INSERT INTO Contacts_Entreprises (
                                            entreprise_id, type, valeur, sources, status
                                        )
                                        VALUES (%s, %s, %s, %s, %s);
                                    '''

                                    data_tuple_contacts_tel2 = (entreprise_id, type_contact_tel2, valeur_tel2, source, True)
                                    cursor.execute(insert_query_contacts_tel2, data_tuple_contacts_tel2)

                                # Insert data into the Contacts_Entreprises table for email addresses
                                if est_email(valeur_email):
                                    insert_query_contacts_email = '''
                                        INSERT INTO Contacts_Entreprises (
                                            entreprise_id, type, valeur, sources, status
                                        )
                                        VALUES (%s, %s, %s, %s, %s);
                                    '''

                                    data_tuple_contacts_email = (entreprise_id, 'Mail', valeur_email, source, True)
                                    cursor.execute(insert_query_contacts_email, data_tuple_contacts_email)

                    conn.commit()
                    print("Data inserted successfully.")

                except Exception as e:
                    conn.rollback()
                    print(f"Error inserting data: {e}")

    except psycopg2.Error as e:
        print(f"Error connecting to the database: {e}")

# Example usage of the function to insert data from a folder of Excel files
excel_folder_path = '../../DONNEES FICHIERS/FICHIERSWM'
insert_data_from_excel_folder(excel_folder_path) 

Inserting data from WM-Base de prospection V3 29-11-230.xlsx...
Error inserting data: ERREUR:  syntaxe en entrée invalide pour le type timestamp : « NaT »
LINE 5: ...                                   VALUES ('nat', 'NaT'::tim...
                                                             ^

